### Importing libraries

In [2]:
import pandas as pd

### Importing dataset

In [4]:
minors_master = pd.read_csv("data/obts/minors_sentences_master.csv")

columns_to_drop = ["MESSAGE_TYPE", "ARREST_NAME", "FINAL_NAME", "PR_FLST_CHAP", "PR_FLST_SECT", "PR_FLST_SUBSECT",
                  "CT_FLST_CHAP", "CT_FLST_SECT", "CT_FLST_SUBSECT", "ST_PROB_LEN", "ST_COMM_LEN", "ST_FINE",
                   "ST_COURT_COST", "ST_RESTITUTION"]
minors_master = minors_master.drop(columns = columns_to_drop)

minors_master.head()

C:\Users\sdasgupta\AppData\Local\Temp\ipykernel_20036\696628784.py:1: DtypeWarning: Columns (2,23,24,27,29,34,35,43,47,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  minors_master = pd.read_csv("data/obts/minors_sentences_master.csv")


,ID,OFF_TRAN_NO,COURT_DOCKET_NO,SEQUENCE_NO,NO_OF_COUNTS,COURT_DESIGNATOR,COUNTY,SEX,RACE,BIRTH_DATE,...,CT_TRIAL_TYPE,CT_FINAL_PLEA,ST_IMPOSED_DT,ST_CONFINE,ST_CONFINE_TYPE,ST_CONFINE_LEN_SUSP,ST_CREDIT_TIME,ST_SENT_PROV,ST_SP_SENT_PROV,ST_IMPOSED_YR
0,48,8888888888,012005CF005485XXXAXX,1,1,R,1,M,W,1987-10-15,...,NaN,NaN,NaN,NaN,N,888888.0,0.0,NNNNN,NNNNNNN,NaN
1,49,8888888888,012005CF005485XXXAXX,2,1,R,1,M,W,1987-10-15,...,NaN,NaN,NaN,NaN,N,888888.0,0.0,NNNNN,NNNNNNN,NaN
2,50,8888888888,012005CF005485XXXAXX,3,1,R,1,M,W,1987-10-15,...,NaN,NaN,NaN,NaN,N,888888.0,0.0,NNNNN,NNNNNNN,NaN
3,51,8888888888,012005CF005485XXXAXX,4,1,R,1,M,W,1987-10-15,...,NaN,NaN,NaN,NaN,N,888888.0,0.0,NNNNN,NNNNNNN,NaN
4,290,8888888888,012007CF002605XXXAXX,1,1,R,1,M,B,1989-06-15,...,NaN,NaN,NaN,NaN,N,888888.0,0.0,NNNNN,NNNNNNN,NaN


In [5]:
minors_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278795 entries, 0 to 278794
Data columns (total 41 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    278795 non-null  int64  
 1   OFF_TRAN_NO           278795 non-null  object 
 2   COURT_DOCKET_NO       278795 non-null  object 
 3   SEQUENCE_NO           278795 non-null  int64  
 4   NO_OF_COUNTS          278795 non-null  int64  
 5   COURT_DESIGNATOR      278795 non-null  object 
 6   COUNTY                278795 non-null  int64  
 7   SEX                   276439 non-null  object 
 8   RACE                  273320 non-null  object 
 9   BIRTH_DATE            278795 non-null  object 
 10  INIT_ARREST_DT        274971 non-null  object 
 11  CLERK_DT              278782 non-null  object 
 12  OFFENSE_DT            278795 non-null  object 
 13  CAPIAS_DT             74337 non-null   object 
 14  REARREST_DT           31925 non-null   object 
 15  

In [6]:
minors_master["CT_CHARGE_LEVEL"].value_counts()

CT_CHARGE_LEVEL
M    69523
F    66357
N    49469
I    37669
C     4472
L     2940
3        1
Name: count, dtype: int64

### Analyzing data

#### Felonies and misdemeanors

In [9]:
### filters for felonies and misdemeanors
data = minors_master[minors_master["CT_CHARGE_LEVEL"].isin(["M", "F"])].reset_index(drop = True)

### stores offenders with sentences of confinement
minors_master_sent = data[data["ST_CONFINE"].notna()].reset_index(drop = True)
### creating list of unique offenders
unique_sent = minors_master_sent["OFF_TRAN_NO"].unique().tolist()

### stores offenders with juvenile sanctions
minors_master_juvsent = data[data["CT_ACTION_TAKEN"] == "K"].reset_index(drop = True)

### stores offenders with no sentences of confinement
minors_master_nosent = data[data["ST_CONFINE"].isna()].reset_index(drop = True)
### removing offenders whose other charges received a sentence of confinement
minors_master_nosent = minors_master_nosent[~minors_master_nosent["OFF_TRAN_NO"].isin(unique_sent)].reset_index(drop = True)

### stores offenders with life sentences
minors_master_life = data[data["ST_CONFINE"] == "LIFE"].reset_index(drop = True)
### stores offenders with death sentences
minors_master_death = data[data["ST_CONFINE"] == "DEATH"].reset_index(drop = True)

print("Total minors:", data["OFF_TRAN_NO"].nunique())
print("Minors with sentences of confinement:" , minors_master_sent["OFF_TRAN_NO"].nunique())
print("Minors with juvenile sanctions:" , minors_master_juvsent["OFF_TRAN_NO"].nunique())
print("Minors with no sentences of confinement:", minors_master_nosent["OFF_TRAN_NO"].nunique())
print("Minors with life sentences:", minors_master_life["OFF_TRAN_NO"].nunique())
print("Minors with death sentences:", minors_master_death["OFF_TRAN_NO"].nunique())


Total minors: 33841
Minors with sentences of confinement: 18407
Minors with juvenile sanctions: 2631
Minors with no sentences of confinement: 15434
Minors with life sentences: 44
Minors with death sentences: 1


In [10]:
data["COURT_DESIGNATOR"].value_counts()

COURT_DESIGNATOR
R    85838
C    50042
Name: count, dtype: int64

#### Felonies

In [12]:
### filters for felonies
data = minors_master[minors_master["CT_CHARGE_LEVEL"] == "F"].reset_index(drop = True)

### stores offenders with sentences of confinement
minors_master_sent = data[data["ST_CONFINE"].notna()].reset_index(drop = True)
### creating list of unique offenders
unique_sent = minors_master_sent["OFF_TRAN_NO"].unique().tolist()

### stores offenders with juvenile sanctions
minors_master_juvsent = data[data["CT_ACTION_TAKEN"] == "K"].reset_index(drop = True)

### stores offenders with no sentences of confinement
minors_master_nosent = data[data["ST_CONFINE"].isna()].reset_index(drop = True)
### removing offenders whose other charges received a sentence of confinement
minors_master_nosent = minors_master_nosent[~minors_master_nosent["OFF_TRAN_NO"].isin(unique_sent)].reset_index(drop = True)

### stores offenders with life sentences
minors_master_life = data[data["ST_CONFINE"] == "LIFE"].reset_index(drop = True)
### stores offenders with death sentences
minors_master_death = data[data["ST_CONFINE"] == "DEATH"].reset_index(drop = True)

print("Total minors:", data["OFF_TRAN_NO"].nunique())
print("Minors with sentences of confinement:" , minors_master_sent["OFF_TRAN_NO"].nunique())
print("Minors with juvenile sanctions:" , minors_master_juvsent["OFF_TRAN_NO"].nunique())
print("Minors with no sentences of confinement:", minors_master_nosent["OFF_TRAN_NO"].nunique())
print("Minors with life sentences:", minors_master_life["OFF_TRAN_NO"].nunique())
print("Minors with death sentences:", minors_master_death["OFF_TRAN_NO"].nunique())

Total minors: 22812
Minors with sentences of confinement: 15590
Minors with juvenile sanctions: 1953
Minors with no sentences of confinement: 7222
Minors with life sentences: 44
Minors with death sentences: 1


In [13]:
data["COURT_DESIGNATOR"].value_counts()

COURT_DESIGNATOR
R    66353
C        4
Name: count, dtype: int64

#### Misdemeanors

In [15]:
### filters for misdemeanors
data = minors_master[minors_master["CT_CHARGE_LEVEL"] == "M"].reset_index(drop = True)

### stores offenders with sentences of confinement
minors_master_sent = data[data["ST_CONFINE"].notna()].reset_index(drop = True)
### creating list of unique offenders
unique_sent = minors_master_sent["OFF_TRAN_NO"].unique().tolist()

### stores offenders with juvenile sanctions
minors_master_juvsent = data[data["CT_ACTION_TAKEN"] == "K"].reset_index(drop = True)

### stores offenders with no sentences of confinement
minors_master_nosent = data[data["ST_CONFINE"].isna()].reset_index(drop = True)
### removing offenders whose other charges received a sentence of confinement
minors_master_nosent = minors_master_nosent[~minors_master_nosent["OFF_TRAN_NO"].isin(unique_sent)].reset_index(drop = True)

### stores offenders with life sentences
minors_master_life = data[data["ST_CONFINE"] == "LIFE"].reset_index(drop = True)
### stores offenders with death sentences
minors_master_death = data[data["ST_CONFINE"] == "DEATH"].reset_index(drop = True)

print("Total minors:", data["OFF_TRAN_NO"].nunique())
print("Minors with sentences of confinement:" , minors_master_sent["OFF_TRAN_NO"].nunique())
print("Minors with juvenile sanctions:" , minors_master_juvsent["OFF_TRAN_NO"].nunique())
print("Minors with no sentences of confinement:", minors_master_nosent["OFF_TRAN_NO"].nunique())
print("Minors with life sentences:", minors_master_life["OFF_TRAN_NO"].nunique())
print("Minors with death sentences:", minors_master_death["OFF_TRAN_NO"].nunique())

Total minors: 17096
Minors with sentences of confinement: 5264
Minors with juvenile sanctions: 1223
Minors with no sentences of confinement: 11832
Minors with life sentences: 0
Minors with death sentences: 0


In [16]:
data["COURT_DESIGNATOR"].value_counts()

COURT_DESIGNATOR
C    50038
R    19485
Name: count, dtype: int64